# Project 2, part 1 - SoC-FPGA Digital Pulse Processing system and TCL
This Jupyter Notebook (JN) is the continuation of the Lab guide that will take you through the necessary steps to interact with the ZedBoard development board via Ethernet using the UDMA framework. All the provided source code is written in Python 3.

In this Lab exercise you will not only interact with the design in the Zynq SoC, but also with the external DAC and ADC boards plugged into the ZedBoard PMOD connectors. To do so, we have prepared an environment based on Python 3.8 and the required libraries to execute the workflow described next:
 
1) **Python + UDMA <---> Zynq SoC**
  - Send from this JN to the ComBlock's FIFO a
  - Use the UDMA library to write individual ComBlock output registers to setup the ADC and DAC driver values
  - Use the UDMA library to read the contents of the data stored in the ComBlock's FIFO
3) **Python**
  - Assess the contents of the data read from the ComBlock's FIFO with the expected behavior of the FIR filter within the programmable logic design in the SoC

All the required steps are documented in this JN, so you can easily follow them along with the code you execute in the notebook blocks.

- Author(s): Bruno Valinoti (MLab/ICTP) - 2023/10/31 (version 1.0)
- Update(s):

# 1) Generate the project with a TCl
Before starting with this JN, be sure you configured the FPGA and programmed the PS with the Light DPP project. 

---

# 2) Interfacing with hardware via UDMA
In this section you will make use of the synthesized waveforms from the last section to periodically execute the following loop:

- Transmit a synthesized waveform to the ComBlock's RAM using the UDMA.
- The hardware design in the PL of the SoC will continuously send these samples to the DAC (plugged into the ZedBoard's PMOD connector), so they get translated to the analog domain.
- This analog signal will be continuously sampled by the ADC (also plugged into the ZedBoard PMOD connector).
- The sampled values from the ADC stream will feed the input of the finite impulse response (FIR) filter that was generated with High-Level Synthesis (HLS).
- The output of the FIR filter will be stored in the ComBlock's input FIFO.
- The data stored in the FIFO will be sent from the SoC to this computer using UDMA.

## Setting up UDMA and ZedBoard parameters

Import libraries

In [ ]:
# Setting up plot rendering handler for Jupyter Notebook
%matplotlib widget

# == MLab libraries ==
import udma #MLab UDMA library

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import time

IP Address and port of your ZedBoard development board

In [ ]:
IP_ADDRESS = '192.168.1.10' # Set your ZedBoard IP Address here
IP_PORT = 7

Initializing UDMA class instance with provided IP settings

In [ ]:
zedBoard = udma.UDMA_CLASS(IP_ADDRESS, IP_PORT)

### Connecting to ZedBoard

Tries indefinitely until connection is successful

In [ ]:
connectionStatus = 0
while(connectionStatus == 0):
    connectionStatus = zedBoard.connect()
    time.sleep(1)

In [ ]:
# Disable unused logging to speed up data transactions
_ = zedBoard.log(0)

# Interacting with ZedBoard via UDMA

## Send the square pulse signal and process the data using the DPP for amplitude extraction
Dataflow (in SoC PL) to process the data:
- Disable DPP operation
- Setup the detection threshold level
- Setup the operation mode 
- Clear FIFOs
- Send data to the FIFO output of the ComBlock
- Enable DPP operation
- Processed data captured in Comblock's input FIFO
- Data from ComBlock's FIFO are fetched via UDMA to this Jupyter Notebook
- Verify the pulses amplitude according to the threshold level slected up before

In [ ]:
# Function definition: used to clear out FIFO contents prior to write or read any value
FIFO_CLEAR_REG = 1
def clearFifos(udmaInstance):
    udmaInstance.write_reg(FIFO_CLEAR_REG, 1)
    udmaInstance.write_reg(FIFO_CLEAR_REG, 0)

### Writo into Comblock's FIFO using UDMA and process the pulses 

Here you will send a series of pulses to the DPP using the Comblock's input FIFO, enable the operation and fetch the data processed by a light dpp output using the Comblock output FIFO. 
Execute this block as many times as you want (```Ctrl+Enter```) to observe the results in the time domain.

In [ ]:
ENA_OPERATION_REG  = 0
FIFO_CLEAR_REG     = 1
THRESHOLD_HIGH_REG = 2
OP_MODE_REG        = 3


pulse = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 68, 68, 68, 68, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 350, 350, 350, 350, 350, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 653, 653, 653, 653, 653, 653, 653, 653, 653, 653, 653, 653, 653, 653, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 264, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 94, 94, 94, 94, 94, 94, 94, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1038, 1038, 1038, 1038, 1038, 1038,
                  1038, 1038, 1038, 1038, 1038, 1038, 1038, 1038, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 28, 28, 28, 28, 28, 28, 28, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

# Clear input FIFO contents to get the latest data from the FIR filter output
zedBoard.write_reg(ENA_OPERATION_REG, 0x00) # Disable DPP operation

# Set threshold value
zedBoard.write_reg(THRESHOLD_HIGH_REG, 200)

# Set operation mode
zedBoard.write_reg(OP_MODE_REG, 0x00)

clearFifos(zedBoard)
# Empty list declaration: the data read from the ComBlock's FIFO will be stored here
fifoInputTrace = []

#Send data to FIFO
zedBoard.write_fifo(len(pulse),pulse)
zedBoard.write_reg(ENA_OPERATION_REG, 0x01) # Enable DPP operation
time.sleep(1)

fifoSamples = zedBoard.read_fifo(10)[1]

print(fifoSamples)

# Matching the numerical representation of the FIR filter output (integer 16 bits)
fifoSamples = np.array(fifoSamples).astype(np.int16)
    
# Appending to the result array the latest data read from FIFO
fifoInputTrace.extend(fifoSamples)

# Plotting the Histogram of the pulses amplitude
fig = plt.figure()

plt.hist(fifoInputTrace, bins=30, edgecolor='black')  # Adjust the number of bins as needed
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Histogram of Pulse Data')
plt.show()
